In [2]:
import torch.nn as nn
import pandas as pd
import numpy as np
import sklearn
import tqdm

In [3]:
import os
import replicate

In [4]:
# Llama2
class LLM1():
    def __init__(self, api):
        #Set the REPLICATE_API_TOKEN environment variable
        os.environ["REPLICATE_API_TOKEN"] = api
        self.model_version = "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3"
    
    def run(self, prompt):
        output = replicate.run(
            self.model_version,
            input={
                "prompt": prompt
            }
        )
        response = ""
        for word in output:
            response += word
        return response


api = "r8_Z3yjMbxweBHOIDzKuDLtCGgkpXlGHmI2yWxXk"

In [5]:
from transformers import AutoTokenizer, AutoModel

# ChatGLM
class LLM2():
    def __init__(self, address):
        self.tokenizer = AutoTokenizer.from_pretrained(address, trust_remote_code=True)
        self.model = AutoModel.from_pretrained(address, trust_remote_code=True).quantize(4).half().cuda()
        self.model = self.model.eval()

    def run(self, prompt):
        response, history = self.model.chat(self.tokenizer, prompt, history = [])
        return response


address = "C:\\Users\\By Yu\\chatglm-6b"

c:\Users\jackl\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class Toolkit:
    def __init__(self, tools):
        # tools is a list of three-tuples of (name, function, description)
        # self.tools is a dictionary from names to two tuples of (function, description)
        self.tools = {}
        for item in tools:
            if item[0] in self.tools.keys():
                print("Tool \"{}\" already exist.".format(name))
                continue
            tools[item[0]] = (item[1], item[2])
    
    def __str__(self):
        string = ""
        for tool in self.tools.items():
            string += tool[0]
            string += ": "
            string += tool[1][0].__name__
            string += ", "
            string += tool[1][1]
            string += '\n'
        return string
    
    def add_tool(self, new_tool):
        name = new_tool[0]
        if (name == ""):
            name = new_tool[1].__name__
        if name in self.tools.keys():
            print("Tool \"{}\" already exist.".format(name))
            return -1
        self.tools[name] = (new_tool[1], new_tool[2])
    
    def remove_tool(self, name):
        if name in self.tools.keys():
            return self.tools.pop(name)
        else:
            print("Tool {} not exist.".format(name))
    
    def call_tool(self, name, tool_inputs):
        if name in self.tools.keys():
            return self.tools[name][0](tool_inputs)
        else:
            print("Tool {} not exist.".format(name))
    
    

In [30]:
class Agent:
    def __init__(self, pre_prompt, model, toolkit, hyperparameters = {}):
        self.pre_prompt = pre_prompt # string, starting lines of the agent, role and purpose
        self.model = model # large language model, (supposedly) a class that encapsulates a language model to regulate the interface
        self.toolkit = toolkit # a class that manages all the tools and descriptions
        self.thought = [] # the current thought, a list is to align with action and observation
        self.action = [] # the current action, a list for many tool-callings
        self.observation = [] # the current observation, a list for many tool-callings
        # memory, a dictionary of previous thoughts, actions and observations
        self.memory = {"thought": [], "action": [], "observation": []} # should be linked list
        # how many memory to record, weight of each input material
        self.hyperparameters = hyperparameters
    
    def add_tool(self, new_tool):
    # add a new tool to toolkit
    # tool contains a function and its description
    # name is default the function name
        Toolkit.add_tool(new_tool)
    
    def remove_tool(self, name):
    # remove an exsisting tool from toolkit by its name
    # return the removed tool
        return toolkit.remove_tool(name)
    
    def call_tool(self, name, tool_inputs):
    # call a tool by its name and the given inputs
        return toolkit.call_tool(name, tool_inputs)
    
    def generate_prompt(self, inputs):
    # generate the prompt based on, pre-prompt, input information, tool descriptions, and memory
        prompt = ""
        return prompt
    
    def update_memory(self):
        self.memory["thought"].append(self.thought)
        self.memory["action"].append(self.action)
        self.memory["observation"].append(self.observation)
        # delete previous memory if too much
        for i in range(len(self.memory["thought"]) - self.hyperparameters["max_memory_len"]):
            self.memory["thought"].pop(0)
            self.memory["action"].pop(0)
            self.memory["observation"].pop(0)
    
    def Thought(self, inputs):
    # determine the action based on input information and tool descriptions
    # input the prompt to model
        prompt = self.generate_prompt(inputs)
        response = self.model.run(prompt)
        self.thought = []
        self.thought.append(response)
        new_action = ("Tool name", "Tool input") # action name and input need to be extracted
        self.action = []
        self.action.append(new_action)
    
    def Action(self):
    # act on the decision made in thought
    # get observation (tool output)
        self.observation = []
        # self.observation.append(call_tool(self.action[-1][0], self.action[-1][1]))
        self.observation.append("Tool output")
        
    def Observation(self):
    # evaluate the observation, from users
        return
    
    def output(self):
        string = "Current storage:\nThought: " + self.thought[0]
        string += "\n\nAction: " + self.action[0][0] + ", " + self.action[0][1]
        string += "\n\nObservation: " + self.observation[0]
        return string
    
    def run(self, inputs):
        self.Thought(inputs)
        self.Action()
        self.Observation()
        self.update_memory()
        

In [7]:
def main():
    api = "r8_Z3yjMbxweBHOIDzKuDLtCGgkpXlGHmI2yWxXk"
    model = LLM1(api)
    tools = [] # a list of three-tuples of (name, function, description)
    toolkit = Toolkit(tools)
    pre_prompt = """xxx"""
    hyperparameters = {"max_memory_len": 2, "prompt_memory_len": 1}
    agent = Agent(pre_prompt, model, toolkit, hyperparameters)
    agent.run("Testing")
    string = agent.output()
    print(string)
    

In [32]:
main()

Current storage:
Thought:  Hello! I'm happy to assist you in any way I can. Is there something specific you need help with today? Do you have a task or project you'd like me to assist you with, or is there something else I can help you with? Let me know and I'll do my best to be of service!

Action: Tool name, Tool input

Observation: Tool output
